# Funções Diversas (descrição)

### `plota_ppi`: Gera gráfico de uma elevação com shapefile de São Paulo

Parâmetros:
* `arquivo` - arquivo com os dados de radar (já lido pelo Py-ART)
* `variavel` - variável a ser plotada
* `var_min` - valor mínimo da variável na escala do plot
* `var_max` - valor máximo da variável na escala do plot
* `elevacao` - elevação a ser plotada
* `lon_lims` - limites de longitude do plot
* `lat_lims` - limites de latitude do plot

In [1]:
def plota_ppi(arquivo, elevacao, lon_lims, lat_lims, 
             variavel = 'corrected_reflectivity', var_min = 10, var_max = 70):
    
    display = pyart.graph.RadarMapDisplay(arquivo)
    fig = plt.figure(figsize = [12,9])
    display.plot_ppi_map(variavel, sweep = elevacao, shapefile = "shapefiles/sao_paulo", vmin = var_min, vmax = var_max, 
                         min_lat = lat_lims[0], max_lat = lat_lims[1], min_lon = lon_lims[0], max_lon = lon_lims[1], 
                         lat_lines = np.arange(lat_lims[0], lat_lims[1], .5), 
                         lon_lines = np.arange(lon_lims[0], lon_lims[1], .5))

### `two_panel_plot`: Gera painel com dois gráficos com shapefile de São Paulo

Parâmetros:
* `radar` - arquivo com os dados de radar (já lido pelo Py-ART)
* `lon_lims` - limites de longitude dos plots
* `lat_lims` - limites de latitude dos plots
* `sweep` - elevação a ser plotada
* `var1` - variável a ser plotada no gráfico da esquerda
* `vmin1` - valor mínimo da variável da esquerda na escala do plot
* `vmax1` - valor máximo da variável da esquerda na escala do plot
* `cmap1` - escala de cores do gráfico da esquerda
* `units1` - unidade da escala do gráfico da esquerda
* `var2` - variável a ser plotada no gráfico da direita
* `vmin2` - valor mínimo da variável da direita na escala do plot
* `vmax2` - valor máximo da variável da direita na escala do plot
* `cmap2` - escala de cores do gráfico da direita
* `units2` - unidade da escala do gráfico da direita
* `return_flag` - painel deve ser plotado? Defaut: False

In [2]:
def two_panel_plot(radar, lon_lims, lat_lims, sweep=0, var1='reflectivity', vmin1=10, vmax1=70,
                   cmap1='RdYlBu_r', units1='dBZ', var2='differential_reflectivity',
                   vmin2=-5, vmax2=5, cmap2='RdYlBu_r', units2='dB', return_flag=False):
    
    display = pyart.graph.RadarDisplay(radar)
    fig = plt.figure(figsize=(13,5))
    
    ax1 = fig.add_subplot(121)
    display.plot_ppi_map(var1, sweep = sweep, shapefile = "shapefiles/sao_paulo",
                         vmin = vmin1, vmax = vmax1, cmap = cmap1, colorbar_label = units1, mask_outside = True,
                         min_lat = lat_lims[0], max_lat = lat_lims[1], min_lon = lon_lims[0], max_lon = lon_lims[1],
                         lat_lines = np.arange(lat_lims[0], lat_lims[1], .5), 
                         lon_lines = np.arange(lon_lims[0], lon_lims[1], .5))
    
    ax2 = fig.add_subplot(122)
    display.plot_ppi_map(var2, sweep = sweep, shapefile = "shapefiles/sao_paulo",
                         vmin = vmin2, vmax = vmax2, cmap = cmap2, colorbar_label = units2, mask_outside = True,
                         min_lat = lat_lims[0], max_lat = lat_lims[1], min_lon = lon_lims[0], max_lon = lon_lims[1],
                         lat_lines = np.arange(lat_lims[0], lat_lims[1], .5), 
                         lon_lines = np.arange(lon_lims[0], lon_lims[1], .5))
    
    if return_flag:
        return fig, ax1, ax2, display

### `radar_coords_to_cart`: Convertendo coordenadas do radar para cartesianas

TJL - taken from old Py-ART version

Calculate Cartesian coordinate from radar coordinates

__Parameters__
* `rng`: array - Distances to the center of the radar gates (bins) in kilometers.
* `az`: array - Azimuth angle of the radar in degrees.
* `ele`: array -Elevation angle of the radar in degrees.

__Returns__
* `x, y, z`: array - Cartesian coordinates in meters from the radar.

__Notes__
The calculation for Cartesian coordinate is adapted from equations 2.28(b) and 2.28(c) of Doviak and Zrnic [1]_ assuming a standard atmosphere (4/3 Earth's radius model).

\begin{equation*}
z = \sqrt{r^2+R^2+r*R*sin(\theta_e)} - R  \\
s = R * arcsin(\frac{r*cos(\theta_e)}{R+z})  \\
x = s * sin(\theta_a)  \\
y = s * cos(\theta_a)  \\
\end{equation*}

Where r is the distance from the radar to the center of the gate, $\theta_a$ is the azimuth angle, $\theta_e$ is the elevation angle, $s$ is the arc length, and $R$ is the effective radius of the earth, taken to be 4/3 the mean radius of earth (6371 km).

__References__

[1] Doviak and Zrnic, Doppler Radar and Weather Observations, Second Edition, 1993, p. 21.

In [3]:
def radar_coords_to_cart(rng, az, ele, debug=False):

    theta_e = ele * np.pi / 180.0  # elevation angle in radians.
    theta_a = az * np.pi / 180.0  # azimuth angle in radians.
    R = 6371.0 * 1000.0 * 4.0 / 3.0  # effective radius of earth in meters.
    r = rng * 1000.0  # distances to gates in meters.

    z = (r ** 2 + R ** 2 + 2.0 * r * R * np.sin(theta_e)) ** 0.5 - R
    s = R * np.arcsin(r * np.cos(theta_e) / (R + z))  # arc length in m.
    x = s * np.sin(theta_a)
    y = s * np.cos(theta_a)
    return x, y, z

### `get_z_from_radar`: Calculando altura do radar correspondente às elevações

Input radar object, return z from radar (km, 2D)

In [4]:
def get_z_from_radar(radar):
    azimuth_1D = radar.azimuth['data']
    elevation_1D = radar.elevation['data']
    srange_1D = radar.range['data']
    sr_2d, az_2d = np.meshgrid(srange_1D, azimuth_1D)
    el_2d = np.meshgrid(srange_1D, elevation_1D)[1]
    xx, yy, zz = radar_coords_to_cart(sr_2d/1000.0, az_2d, el_2d)
    return zz + radar.altitude['data']

### `check_sounding_for_montonic`: Forçando os dados de sondagem a serem monotônicos

So the sounding interpolation doesn't fail, force the sounding to behave monotonically so that z always increases. This eliminates data from descending balloons.

In [5]:
def check_sounding_for_montonic(sounding):
    snd_T = sounding.soundingdata['temp']  # In old SkewT, was sounding.data
    snd_z = sounding.soundingdata['hght']  # In old SkewT, was sounding.data
    dummy_z = []
    dummy_T = []
    if not snd_T.mask[0]: #May cause issue for specific soundings
        dummy_z.append(snd_z[0])
        dummy_T.append(snd_T[0])
        for i, height in enumerate(snd_z):
            if i > 0:
                if snd_z[i] > snd_z[i-1] and not snd_T.mask[i]:
                    dummy_z.append(snd_z[i])
                    dummy_T.append(snd_T[i])
        snd_z = np.array(dummy_z)
        snd_T = np.array(dummy_T)
    return snd_T, snd_z

### `interpolate_sounding_to_radar`: interpolando dados de radiossondagens aos de radar

Takes sounding data and interpolates it to every radar gate.

In [6]:
def interpolate_sounding_to_radar(sounding, radar):
    radar_z = get_z_from_radar(radar)
    radar_T = None
    snd_T, snd_z = check_sounding_for_montonic(sounding)
    shape = np.shape(radar_z)
    rad_z1d = radar_z.ravel()
    rad_T1d = np.interp(rad_z1d, snd_z, snd_T)
    return np.reshape(rad_T1d, shape), radar_z

### `add_field_to_radar_object`: transformando array em dado de radar

Adds a newly created field to the Py-ART radar object. If reflectivity is a masked array, make the new field masked the same as reflectivity.

In [8]:
def add_field_to_radar_object(field, radar, field_name='fuzzy_hydrometeors', units='unitless', 
                              long_name='Hydrometeor ID from Fuzzy Logic', standard_name='Hydrometeor ID',
                              dz_field='corrected_reflectivity'):
    fill_value = -32768
    masked_field = np.ma.asanyarray(field)
    masked_field.mask = masked_field == fill_value
    if hasattr(radar.fields[dz_field]['data'], 'mask'):
        setattr(masked_field, 'mask', 
                np.logical_or(masked_field.mask, radar.fields[dz_field]['data'].mask))
        fill_value = radar.fields[dz_field]['data'].fill_value
    field_dict = {'data': masked_field,
                  'units': units,
                  'long_name': long_name,
                  'standard_name': standard_name,
                  '_FillValue': fill_value}
    radar.add_field(field_name, field_dict, replace_existing=True)
    return radar

### `